In [20]:
import cv2
import numpy as np
from collections import deque


In [21]:

class Vehiculo:
    def __init__(self, vid, x, y, fi):
        self.id = vid
        self.pos = deque(maxlen=15)
        self.pos.append((x, y, fi))
        self.vels = deque(maxlen=10)
        self.v_suav = 0.0
    
    def actualizar(self, x, y, fi, fps):
        self.pos.append((x, y, fi))
        if len(self.pos) >= 5:
            pos_arr = np.array([(p[0], p[1]) for p in self.pos])
            t_total = (self.pos[-1][2] - self.pos[0][2]) / fps
            if t_total > 0:
                dist_px = np.sum(np.sqrt(np.sum(np.diff(pos_arr, axis=0)**2, axis=1)))
                v_kmh = (dist_px * 0.1 / t_total) * 3.6
                if 5 <= v_kmh <= 200 and (not self.vels or abs(v_kmh - self.vels[-1]) <= 20):
                    self.vels.append(v_kmh)
                    v_prom = np.mean(self.vels)
                    self.v_suav = v_prom if self.v_suav == 0 else 0.2*v_prom + 0.8*self.v_suav
    
    def get_vel(self):
        return self.v_suav if len(self.vels) >= 3 else 0.0
    
    def get_pos(self):
        return self.pos[-1][:2] if self.pos else None
    
    def get_frame(self):
        return self.pos[-1][2] if self.pos else 0

In [22]:
def encontrar_vehiculo(cx, cy, vehiculos, frame_idx, max_dist=50, max_frames=30):
    """Busca el vehículo más cercano dentro de los límites especificados"""
    mejor = None
    dist_min = max_dist ** 2
    
    for vid, v in vehiculos.items():
        p = v.get_pos()
        if p and frame_idx - v.get_frame() <= max_frames:
            dist = (cx - p[0]) ** 2 + (cy - p[1]) ** 2
            if dist < dist_min:
                dist_min = dist
                mejor = vid
    return mejor

def limpiar_vehiculos(vehiculos, velocidades, frame_idx, timeout=30, v_min=5):
    """Elimina vehículos antiguos y guarda sus velocidades"""
    viejos = [vid for vid, v in vehiculos.items() if frame_idx - v.get_frame() > timeout]
    for vid in viejos:
        vel = vehiculos[vid].get_vel()
        if vel >= v_min:
            velocidades.append(vel)
        del vehiculos[vid]

In [23]:
LINEAS = [
    (420, 750, 550, 750), (550, 650, 620, 650), (990, 750, 1090, 750),
    (1030, 700, 1110, 700), (1180, 640, 1210, 640), (1420, 730, 1490, 730),
    (1420, 670, 1490, 670)
]

# Parámetros
UMBRAL, AREA_MIN, AREA_MAX = 30, 1500, 15000
TOLERANCIA, MAX_DIST, TIMEOUT = 30, 50, 30

In [ ]:
vehiculos_activos = [{} for _ in LINEAS]
contadores = [0] * len(LINEAS)
velocidades = [[] for _ in LINEAS]
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

video = cv2.VideoCapture('trafico01.mp4')
fps = video.get(cv2.CAP_PROP_FPS)
frames = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    frames.append(frame)
video.release()

# Calcular fondo
fondo = np.mean([frames[i] for i in range(0, len(frames), max(1, len(frames)//100))], axis=0).astype(np.uint8)

# Máscara ROI
h, w = frames[0].shape[:2]
mascara = np.zeros((h, w), dtype=np.uint8)
cv2.fillPoly(mascara, [np.array([[200,550], [200,900], [1700,900], [1700,550]])], 255)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(1080, 1920), dtype=uint8)

In [25]:
for frame_idx, frame in enumerate(frames):
    # Detección
    dif = cv2.absdiff(frame, fondo)
    gris = cv2.cvtColor(dif, cv2.COLOR_BGR2GRAY)
    _, binario = cv2.threshold(gris, UMBRAL, 255, cv2.THRESH_BINARY)
    binario = cv2.morphologyEx(binario, cv2.MORPH_CLOSE, kernel, iterations=2)
    binario = cv2.morphologyEx(binario, cv2.MORPH_OPEN, kernel, iterations=2)
    binario = cv2.bitwise_and(binario, binario, mask=mascara)
    
    contornos, _ = cv2.findContours(binario, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img = frame.copy()
    
    # Limpiar vehículos antiguos
    if frame_idx % TIMEOUT == 0:
        for i in range(len(LINEAS)):
            limpiar_vehiculos(vehiculos_activos[i], velocidades[i], frame_idx)
    
    # Dibujar líneas
    for i, (x1, y1, x2, y2) in enumerate(LINEAS):
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
        cv2.putText(img, f'C{i+1}: {contadores[i]}', ((x1+x2)//2-30, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    
    # Procesar contornos
    for cnt in contornos:
        area = cv2.contourArea(cnt)
        if AREA_MIN < area < AREA_MAX:
            x, y, w, h = cv2.boundingRect(cnt)
            if 0.5 < w/h < 3.0:
                cx, cy = x + w//2, y + h//2
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
                
                for i, (x1, y1, x2, y2) in enumerate(LINEAS):
                    if x1 <= cx <= x2 and abs(cy - y1) < TOLERANCIA:
                        vid = encontrar_vehiculo(cx, cy, vehiculos_activos[i], frame_idx)
                        if vid is None:
                            nuevo_id = f"v_{frame_idx}_{cx}"
                            vehiculos_activos[i][nuevo_id] = Vehiculo(nuevo_id, cx, cy, frame_idx)
                            contadores[i] += 1
                        else:
                            vehiculos_activos[i][vid].actualizar(cx, cy, frame_idx, fps)
                        break
    
    # Info global
    total = sum(contadores)
    todas_vel = [v for sub in velocidades for v in sub]
    texto = f'Total: {total}'
    if todas_vel:
        texto += f' | Vel. Media: {np.mean(todas_vel):.0f} km/h'
    cv2.putText(img, texto, (250, 90), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
    
    cv2.imshow('Conteo', img)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Guardar velocidades finales
for i in range(len(LINEAS)):
    for v in vehiculos_activos[i].values():
        vel = v.get_vel()
        if vel >= 5:
            velocidades[i].append(vel)

cv2.destroyAllWindows()

In [26]:
print("\n" + "="*60)
print("RESULTADOS FINALES")
print("="*60)

for i in range(len(LINEAS)):
    if velocidades[i]:
        v_arr = np.array(velocidades[i])
        print(f"Carril {i+1}: {contadores[i]:3d} vehículos | "
              f"Vel: {v_arr.mean():.1f} km/h (min: {v_arr.min():.1f}, max: {v_arr.max():.1f})")
    else:
        print(f"Carril {i+1}: {contadores[i]:3d} vehículos | Sin datos de velocidad")

todas_velocidades = [v for sub in velocidades for v in sub]
print("-"*60)
print(f"TOTAL: {sum(contadores)} vehículos")
if todas_velocidades:
    v_global = np.array(todas_velocidades)
    print(f"Velocidad Global: {v_global.mean():.1f} km/h (min: {v_global.min():.1f}, max: {v_global.max():.1f})")
print("="*60)


RESULTADOS FINALES
Carril 1:   3 vehículos | Vel: 53.2 km/h (min: 47.2, max: 64.0)
Carril 2:   0 vehículos | Sin datos de velocidad
Carril 3:   3 vehículos | Sin datos de velocidad
Carril 4:   3 vehículos | Sin datos de velocidad
Carril 5:   1 vehículos | Sin datos de velocidad
Carril 6:   1 vehículos | Sin datos de velocidad
Carril 7:   2 vehículos | Sin datos de velocidad
------------------------------------------------------------
TOTAL: 13 vehículos
Velocidad Global: 53.2 km/h (min: 47.2, max: 64.0)
